In [3]:
using Distributions

In [4]:
type Individual
    m::Int64
    p::Int64
    v_e::Float64
    mu::Float64
    mu_b::Float64
    mu_sigma::Float64
    y::Array{Float64, 1}
    b::Array{Int64, 2}
    x::Array{Float64, 1}
    z::Array{Float64, 1}
    
    Individual(m, p, v_e, mu, mu_b, mu_sigma) = new(m, p, v_e, mu, mu_b, mu_sigma)
end  

type Population
    n::Int
    base_ind::Individual
    pop::Array{Individual, 1}
    
    Population(n, base_ind) = new(n, base_ind)
end

In [94]:
function RandomInd!(ind::Individual, y_mean = 0, g_sigma = ind.mu_sigma)
    ind.y = rand(Normal(y_mean, g_sigma), ind.m)
    ind.b = rand(Bernoulli(), ind.p, ind.m)
    ind.x = ind.b * ind.y
    ind.z = ind.x + rand(Normal(0, ind.v_e), ind.p)
    1
end

function RandomInd(ind::Individual, y_mean = 0, g_sigma = ind.mu_sigma)
    new_ind   = Individual(ind.m, ind.p, ind.v_e, ind.mu, ind.mu_b, ind.mu_sigma)
    new_ind.y = rand(Normal(y_mean, g_sigma), new_ind.m)
    new_ind.b = rand(Bernoulli(), new_ind.p, new_ind.m)
    new_ind.x = new_ind.b * new_ind.y
    new_ind.z = new_ind.x + rand(Normal(0, ind.v_e), new_ind.p)

    new_ind
end

function Population(n::Int, ind::Individual, pop::Array{Individual, 1})
    new_pop = Population(n, ind)
    new_pop.pop = pop
    new_pop
end

import Base.getindex

function getindex(pop::Population, i::Integer)
    Population(1, pop.base_ind, [getindex(pop.pop, i)])
end

function append!(pop::Population, ind::Individual)
    pop.pop = [pop.pop; ind]
    pop.n = length(pop.pop)
    1
end

function append!(pop1::Population, pop2::Population)
    # COMPARE BASE IND!
    pop1.pop = [pop1.pop; pop2.pop]
    pop.n = length(pop1.pop)
    1
end

function join(pop1::Population, pop2::Population)
    # COMPARE BASE IND!
    new_pop = Population(pop1.n + pop2.n, pop1.base_ind)
    new_pop.pop = [pop1.pop; pop2.pop]
    new_pop
end

function getindex(pop::Population, s::UnitRange)
    Population(length(s), pop.base_ind, getindex(pop.pop, s))
end

function RandomPop!(pop, x_mean = 0, g_sigma = ind.mu_sigma)
    pop.pop = Array(Individual, pop.n)
    for i = 1:pop.n
        pop.pop[i] = RandomInd(pop.base_ind)
    end
    1
end

function mutation!(ind::Individual)
    mutation_y = rand(Binomial(ind.m, ind.mu))
    if(mutation_y > 0)
        for k = 1:mutation_y
            i = rand(DiscreteUniform(1, ind.m))
            ind.y[i] += rand(Normal(0, ind.mu_sigma))
        end
    end
    mutation_b = rand(Binomial(ind.m * ind.p, ind.mu_b))
    if(mutation_b > 0)
        for k = 1:mutation_b
            i = rand(DiscreteUniform(1, ind.p))
            j = rand(DiscreteUniform(1, ind.m))
            ind.b[i,j] = ind.b[i, j] == 1 ? 0 : 1
        end
    end
end

function mutation!(pop::Population)
    for k = 1:pop.n
        mutation!(pop.pop[k])
    end
end

RandomPop! (generic function with 3 methods)

In [101]:
ind = RandomInd(Individual(10, 2, 0.2, 0.5, 0.1, 0.8))



mutation! (generic function with 2 methods)

In [104]:
pop

Population(5000,Individual(10,2,0.2,0.5,0.1,0.8,[-0.6429106964468728,0.6662210573118852,-0.044831957108434184,0.4488651318260003,-0.12745128210107032,1.2047875737182425,-0.24320183143273455,1.1425396632627425,0.036570932532188904,1.1023155140127772],2x10 Array{Int64,2}:
 0  1  0  0  0  1  1  1  1  1
 1  0  0  1  1  1  0  1  1  0,[3.0981671016339387,0.9802179657851481],[3.381315242253108,1.16310497021394]),[Individual(10,2,0.2,0.5,0.1,0.8,[1.2213651127858285,0.1384936841521442,-0.9714043064829808,0.46330006935824697,-0.1847774820308334,-0.8296479204013189,0.40052675588281406,-0.374350707266935,1.6886905485381771,0.20313231782285557],2x10 Array{Int64,2}:
 1  1  1  1  1  1  1  1  1  0
 0  1  1  1  1  0  1  0  1  1,[-0.7687531868851031,0.8042954212093578],[-0.5258011055150882,0.742229622735759]),Individual(10,2,0.2,0.5,0.1,0.8,[1.0961019708907214,1.5579878809257923,-0.3391213839867353,-1.440817646720749,0.2721909489569364,1.2181439110516923,0.8128906743700425,1.3128166861453145,-1.13879595

In [105]:
mutation!(pop)
pop

Population(5000,Individual(10,2,0.2,0.5,0.1,0.8,[-0.6429106964468728,0.6662210573118852,-0.044831957108434184,0.4488651318260003,-0.12745128210107032,1.2047875737182425,-0.24320183143273455,1.1425396632627425,0.036570932532188904,1.1023155140127772],2x10 Array{Int64,2}:
 0  1  0  0  0  1  1  1  1  1
 1  0  0  1  1  1  0  1  1  0,[3.0981671016339387,0.9802179657851481],[3.381315242253108,1.16310497021394]),[Individual(10,2,0.2,0.5,0.1,0.8,[1.2213651127858285,-0.6725058721420705,-2.575164924551209,0.46330006935824697,-0.1847774820308334,-1.439859427266605,0.40052675588281406,0.6044966968681416,1.6886905485381771,0.20313231782285557],2x10 Array{Int64,2}:
 1  0  1  1  1  1  1  1  1  0
 0  1  1  1  1  0  1  0  0  1,[-0.7687531868851031,0.8042954212093578],[-0.5258011055150882,0.742229622735759]),Individual(10,2,0.2,0.5,0.1,0.8,[-0.01671395079435234,1.5579878809257923,1.4114619281682572,-1.440817646720749,0.2721909489569364,2.1285197184932003,1.3844427642137456,2.0539773843018603,-1.13879595

In [50]:
ind = Individual(1000, 10, 0.2, 0.005, 0.001, 0.8)

Individual(1000,10,0.2,0.005,0.001,0.8,#undef,#undef,#undef,#undef)

In [56]:
pop = Population(5000, ind)
RandomPop!(pop)

Population(5000,Individual(10,2,0.2,0.5,0.1,0.8,[-0.6429106964468728,0.6662210573118852,-0.044831957108434184,0.4488651318260003,-0.12745128210107032,1.2047875737182425,-0.24320183143273455,1.1425396632627425,0.036570932532188904,1.1023155140127772],2x10 Array{Int64,2}:
 0  1  0  0  0  1  1  1  1  1
 1  0  0  1  1  1  0  1  1  0,[3.0981671016339387,0.9802179657851481],[3.381315242253108,1.16310497021394]),[Individual(10,2,0.2,0.5,0.1,0.8,[0.5079065345130289,0.5429267537555845,-0.9714043064829808,-1.0245934853775418,-0.1847774820308334,-0.8296479204013189,0.40052675588281406,-0.03933795714517451,1.3699162902716053,0.20313231782285557],2x10 Array{Int64,2}:
 1  1  1  1  1  0  1  1  0  0
 1  1  1  1  1  0  1  1  1  1,[-0.7687531868851031,0.8042954212093578],[-0.5258011055150882,0.742229622735759]),Individual(10,2,0.2,0.5,0.1,0.8,[1.0961019708907214,-0.7319622794761156,-0.3391213839867353,-1.440817646720749,0.4797215122808002,0.7303060506299347,0.8128906743700425,0.3637491174578428,-0.38081

In [52]:
pop[1]

Population(1,Individual(1000,10,0.2,0.005,0.001,0.8,#undef,#undef,#undef,#undef),[Individual(1000,10,0.2,0.005,0.001,0.8,[1.26443,-0.161343,0.33483,-0.469526,-0.112393,0.056023,-0.758425,-1.52125,-0.660738,-1.06202  …  1.05002,1.89124,-0.628888,0.0448372,0.144406,0.962164,-0.603214,-0.503972,0.401649,0.624092],10x1000 Array{Int64,2}:
 0  0  0  0  1  0  1  0  0  0  0  0  1  …  1  0  1  1  0  1  1  0  1  1  0  0
 1  1  1  0  1  0  1  0  0  1  0  0  1     1  1  0  1  1  0  1  0  0  0  1  0
 0  0  1  0  0  0  1  1  1  0  1  1  1     1  0  1  0  1  1  1  1  1  1  1  1
 0  1  1  1  0  1  0  0  1  1  1  1  0     1  1  1  0  1  1  1  1  0  0  0  1
 0  0  1  1  1  1  1  0  1  0  1  0  1     1  1  1  1  1  0  0  1  1  1  0  1
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  1  0  0  0  0  1  1  1  1  1  1  0
 1  1  1  1  0  0  1  0  0  0  1  0  0     0  1  0  1  1  1  0  0  1  0  1  0
 0  0  1  0  0  0  0  1  0  1  1  1  0     1  1  1  1  0  0  0  0  1  0  1  0
 1  0  0  1  1  0  0  1  0  1  1  0  0  